### Import Libraries

In [15]:
import os
import dotenv
import spotipy
import requests
import pandas as pd
import base64 as bs64
from spotipy.oauth2 import SpotifyOAuth

### Initiate Client

In [12]:
dotenv.load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

client_credentials =  bs64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()


In [18]:
token_url = 'https://accounts.spotify.com/api/token'

headers = {
	'Authorization': f'Basic {client_credentials}'
}

data = {
	'grant_type': 'client_credentials'
}

response = requests.post(token_url, headers=headers, data=data)

if response.status_code == 200:
	access_token = response.json()['access_token']
	print("Access token obtained successfully")
else:
	print("Failed to obtain access token")
	print(response.json())
	exit()

Access token obtained successfully


In [20]:
def get_trending_playlist_data(playlist_id, access_token):
    sp = spotipy.Spotify(auth=access_token)
    
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')
   
    music_data = []
    
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # add more attributes as needed (go through the documentation to know what more you can add)
        }

        music_data.append(track_data)

    # create a pandas dataframe from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df 
    

In [21]:
playlist_id = '1LkzzRb1pLHDZ7c30QDjgE'

music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                                           Track Name  \
0                                  No One Can Stop Us   
1                                            All Good   
2                              Menghilanglah Denganku   
3                                                 DPS   
4                                            Big Bang   
5                                              Dreams   
6                                     You're The Kind   
7                                             Lunatic   
8                                          Sweet Scar   
9                                        Digital Love   
10                                            I AM ME   
11                                           If I Die   
12                              Catch Me (feat. Naaz)   
13                                         Both of Us   
14                                         Invitation   
15                                            Shotgun   
16                             

In [22]:
music_df.to_csv('trending_playlist_data.csv', index=False)